In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter
import joblib

# Cargar y Validar los Datos

In [24]:
# Cargar datos procesados
file_path = '../data/interim/creditcard_balanced.csv' 
data = pd.read_csv(file_path)

In [25]:
# Inspección inicial
print("Dimensiones del dataset:", data.shape)
print("Primeras filas del dataset:")
display(data.head())

Dimensiones del dataset: (568630, 31)
Primeras filas del dataset:


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [26]:
print("Distribución de la variable objetivo (Class):")
if 'Class' in data.columns:
    print(data['Class'].value_counts(normalize=True))
else:
    print("La columna 'Class' no está presente en los datos.")

Distribución de la variable objetivo (Class):
Class
0    0.5
1    0.5
Name: proportion, dtype: float64


# Manejo de Valores Faltantes

In [27]:
# Verificar valores faltantes
print("\nValores faltantes por columna:")
missing_values = data.isnull().sum()
print(missing_values[missing_values > 0])


Valores faltantes por columna:
Series([], dtype: int64)


In [28]:
# Porcentaje de valores faltantes
print("\nPorcentaje de valores faltantes por columna:")
missing_percentage = (missing_values / len(data)) * 100
print(missing_percentage[missing_percentage > 0])


Porcentaje de valores faltantes por columna:
Series([], dtype: float64)


In [29]:
if missing_values.sum() > 0:
    data.fillna(data.mean(), inplace=True)
    print("\nSe imputaron valores faltantes con la media.")
else:
    print("No se encontraron valores faltantes.")

No se encontraron valores faltantes.


# Codificación de Variables Categóricas

In [30]:
categorical_cols = data.select_dtypes(include=['object', 'category']).columns
print("\nColumnas categóricas identificadas:")
print(categorical_cols)


Columnas categóricas identificadas:
Index([], dtype='object')


In [31]:
if len(categorical_cols) > 0:
    # Aplicar One-Hot Encoding
    data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
    print("\nSe aplicó One-Hot Encoding. Dimensiones del dataset después de la codificación:")
    print(data.shape)
else:
    print("No se encontraron columnas categóricas para codificar.")

No se encontraron columnas categóricas para codificar.


# Dividir el Dataset (Train/Test Split)

In [32]:
# Separar características y etiquetas
X = data.drop(columns=['Class'])  
y = data['Class']

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  # Stratify para mantener la proporción de clases
)

In [33]:
print("\nTamaño de los conjuntos:")
print(f"Entrenamiento: {X_train.shape}, Prueba: {X_test.shape}")
print("\nDistribución de clases en el conjunto de entrenamiento:")
print(y_train.value_counts(normalize=True))
print("\nDistribución de clases en el conjunto de prueba:")
print(y_test.value_counts(normalize=True))


Tamaño de los conjuntos:
Entrenamiento: (454904, 30), Prueba: (113726, 30)

Distribución de clases en el conjunto de entrenamiento:
Class
0    0.5
1    0.5
Name: proportion, dtype: float64

Distribución de clases en el conjunto de prueba:
Class
1    0.5
0    0.5
Name: proportion, dtype: float64


# Selección de Características

In [34]:
# Matriz de correlación
correlation_matrix = data.corr()

In [35]:
# Características altamente correlacionadas con la variable objetivo
correlation_target = correlation_matrix['Class'].sort_values(ascending=False)[1:11]
print("\nCaracterísticas más correlacionadas con la variable objetivo:")
print(correlation_target)


Características más correlacionadas con la variable objetivo:
V4     0.741339
V11    0.717036
V2     0.529498
V19    0.288029
V20    0.177626
V21    0.147414
V28    0.102841
V27    0.088156
V26    0.069554
V8     0.063410
Name: Class, dtype: float64


In [36]:
# Identificar características con baja correlación con la variable objetivo
low_corr_features = correlation_matrix['Class'][correlation_matrix['Class'] < 0.01].index
if len(low_corr_features) > 0:
    print("\nCaracterísticas con baja correlación con la variable objetivo:")
    print(low_corr_features)
    print("Considera eliminar estas columnas si no tienen un valor analítico evidente.")
else:
    print("No se encontraron características con baja correlación con la variable objetivo.")


Características con baja correlación con la variable objetivo:
Index(['Time', 'V1', 'V3', 'V5', 'V6', 'V7', 'V9', 'V10', 'V12', 'V13', 'V14',
       'V15', 'V16', 'V17', 'V18', 'V22', 'V23', 'V24'],
      dtype='object')
Considera eliminar estas columnas si no tienen un valor analítico evidente.


In [37]:
# Identificar características altamente correlacionadas entre sí
print("\nCaracterísticas redundantes altamente correlacionadas entre sí:")
redundant_features = []
for col in correlation_matrix.columns:
    high_corr = correlation_matrix[col][correlation_matrix[col] > 0.8].index.drop(col)
    if len(high_corr) > 0:
        redundant_features.append((col, list(high_corr)))

if len(redundant_features) > 0:
    for pair in redundant_features:
        print(f"'{pair[0]}' está altamente correlacionada con {pair[1]}")
else:
    print("No se encontraron características redundantes con alta correlación entre sí.")


Características redundantes altamente correlacionadas entre sí:
'V1' está altamente correlacionada con ['V3', 'V5', 'V7']
'V3' está altamente correlacionada con ['V1', 'V5', 'V7', 'V10']
'V4' está altamente correlacionada con ['V11']
'V5' está altamente correlacionada con ['V1', 'V3', 'V7']
'V7' está altamente correlacionada con ['V1', 'V3', 'V5', 'V10']
'V9' está altamente correlacionada con ['V10']
'V10' está altamente correlacionada con ['V3', 'V7', 'V9', 'V12', 'V16', 'V17', 'V18']
'V11' está altamente correlacionada con ['V4']
'V12' está altamente correlacionada con ['V10', 'V14', 'V16', 'V17', 'V18']
'V14' está altamente correlacionada con ['V12']
'V16' está altamente correlacionada con ['V10', 'V12', 'V17', 'V18']
'V17' está altamente correlacionada con ['V10', 'V12', 'V16', 'V18']
'V18' está altamente correlacionada con ['V10', 'V12', 'V16', 'V17']


In [38]:
# Preselección de características
columns_to_drop = list(low_corr_features)
X_train = X_train.drop(columns=columns_to_drop, errors='ignore')
X_test = X_test.drop(columns=columns_to_drop, errors='ignore')

print("\nDimensiones del dataset después de eliminar características irrelevantes o redundantes:")
print(X_train.shape, X_test.shape)


Dimensiones del dataset después de eliminar características irrelevantes o redundantes:
(454904, 12) (113726, 12)


# Estandarización/Escalado de Variables

In [39]:
# Seleccionar columnas numéricas
numeric_cols = X_train.select_dtypes(include=np.number).columns

# Instanciar el escalador
scaler = StandardScaler()

In [40]:
# Ajustar el escalador con los datos de entrenamiento y transformar
X_train_scaled = X_train.copy()
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])

In [41]:
# Transformar el conjunto de prueba con el mismo escalador
X_test_scaled = X_test.copy()
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [42]:
print("Primeras filas del conjunto de entrenamiento escalado:")
display(X_train_scaled.head())

Primeras filas del conjunto de entrenamiento escalado:


,V2,V4,V8,V11,V19,V20,V21,V25,V26,V27,V28,Amount
225908,-0.103564,-1.135830,-0.008527,-0.895665,0.190010,0.028540,-0.368411,-0.055985,0.424774,0.439732,0.063925,-0.386212
154030,-0.679096,-0.616388,-0.027713,-0.191537,-0.432603,-0.409348,-0.176852,-1.104181,0.928810,-0.135011,-0.243160,-0.355758
465602,0.429253,0.554411,0.359145,0.676642,1.295175,0.162514,0.271004,0.130874,-0.502920,0.241509,1.855433,0.026196
276397,-0.261205,-1.060702,0.127156,-0.548150,-0.837447,-0.358095,-0.209916,-1.067787,0.517521,0.194364,0.044782,-0.364680
203835,-0.849219,-0.977542,-0.050253,-0.304897,0.160560,0.208792,-0.019464,-0.318090,-0.354454,-0.088607,-0.087890,0.295433


In [43]:
# Guardar el escalador
scaler_path = "../artifacts/scaler.pkl"
joblib.dump(scaler, scaler_path)
print(f"Escalador guardado en: {scaler_path}")

Escalador guardado en: ../artifacts/scaler.pkl


# Guardar los Datos Procesados

In [44]:
X_train_scaled.to_csv('../data/processed/X_train.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

print("\nConjuntos procesados guardados correctamente.")


Conjuntos procesados guardados correctamente.
